In [8]:
import hopsworks
import pandas as pd
import numpy as np

In [9]:
project = hopsworks.login(
    host="2a5f8040-2d0d-11ed-b5c5-c151c2fe58c1.cloud.hopsworks.ai",
    project="electricity",
    api_key_value="xIGaDiwBZpXxLYRQ.SgheZwqHTZteASnO1PtPMC4HMgco2OVD6VK6xZN3sKCy96FQT27rF0JbBuFivTTA",
)
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://2a5f8040-2d0d-11ed-b5c5-c151c2fe58c1.cloud.hopsworks.ai:443/p/7287
Connected. Call `.close()` to terminate connection gracefully.


In [22]:
# Read local csv file containing holidays
se3_power_gen_df = pd.read_csv("data/Actual Generation per Production Type_202201010000-202301010000(1).csv")

In [23]:
# Remove columns not needed for SE3 - SE3 doesn't use this type of power
se3_power_gen_df.drop(['Area',
                      'Biomass  - Actual Aggregated [MW]',
                      'Fossil Brown coal/Lignite  - Actual Aggregated [MW]',
                      'Fossil Coal-derived gas  - Actual Aggregated [MW]',
                      'Fossil Hard coal  - Actual Aggregated [MW]',
                      'Fossil Oil  - Actual Aggregated [MW]',
                      'Fossil Oil shale  - Actual Aggregated [MW]',
                      'Fossil Peat  - Actual Aggregated [MW]',
                      'Geothermal  - Actual Aggregated [MW]',
                      'Hydro Pumped Storage  - Actual Aggregated [MW]',
                      'Hydro Pumped Storage  - Actual Consumption [MW]',
                      'Hydro Run-of-river and poundage  - Actual Aggregated [MW]',
                      'Other renewable  - Actual Aggregated [MW]',
                      'Waste  - Actual Aggregated [MW]',
                      'Wind Offshore  - Actual Aggregated [MW]'], axis=1, inplace=True)

In [24]:
se3_power_gen_df.rename(columns={c: c.split("-")[0].strip().replace(" ", "_") for c in se3_power_gen_df.columns}, inplace=True)

In [25]:
se3_power_gen_df['zone'] = "SE3"

In [26]:
se3_power_gen_df['MTU'] = pd.to_datetime(se3_power_gen_df['MTU'].apply(lambda x: x.split("-")[0].strip()), format='%d.%m.%Y %H:%M')

In [27]:
for column in [c for c in se3_power_gen_df.columns if c not in ['MTU', 'zone']]:
    se3_power_gen_df[column] = pd.to_numeric(se3_power_gen_df[column], errors="coerce").fillna(0)

In [28]:
se3_power_gen_df[se3_power_gen_df['Solar'] > 0]

,MTU,Fossil_Gas,Hydro_Water_Reservoir,Marine,Nuclear,Other,Solar,Wind_Onshore,zone
9,2022-01-01 09:00:00,0.0,926.0,0.0,5840.0,1028.0,1.0,397.0,SE3
10,2022-01-01 10:00:00,0.0,944.0,0.0,5840.0,1082.0,4.0,307.0,SE3
11,2022-01-01 11:00:00,0.0,953.0,0.0,5884.0,1107.0,7.0,252.0,SE3
12,2022-01-01 12:00:00,0.0,963.0,0.0,6062.0,1111.0,6.0,230.0,SE3
13,2022-01-01 13:00:00,0.0,965.0,0.0,6208.0,1113.0,4.0,221.0,SE3
...,...,...,...,...,...,...,...,...,...
6158,2022-09-14 14:00:00,7.0,464.0,0.0,4574.0,391.0,224.0,934.0,SE3
6159,2022-09-14 15:00:00,17.0,457.0,0.0,4575.0,376.0,180.0,968.0,SE3
6160,2022-09-14 16:00:00,0.0,451.0,0.0,4573.0,367.0,110.0,1024.0,SE3
6161,2022-09-14 17:00:00,0.0,441.0,0.0,4574.0,366.0,52.0,985.0,SE3


In [10]:
se_power_gen_fg = fs.get_or_create_feature_group(
    name="se_power_generation",
    version=1,
    description="Sweden power generation per type",
    primary_key=["zone"],
    event_time="mtu",
    online_enabled=False,
    statistics_config={
        'histograms': True, 
        'correlations': True
    }
)

In [ ]:
se_power_gen_fg.insert(se3_power_gen_df)

Feature Group created successfully, explore it at 
https://2a5f8040-2d0d-11ed-b5c5-c151c2fe58c1.cloud.hopsworks.ai:443/p/7287/fs/7235/fg/14352


Uploading Dataframe: 0.00% |          | Rows 0/8761 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://2a5f8040-2d0d-11ed-b5c5-c151c2fe58c1.cloud.hopsworks.ai/p/7287/jobs/named/se_power_generation_1_offline_fg_backfill/executions


In [41]:
## Data validation
import great_expectations as ge

expectation_suite = ge.core.ExpectationSuite(
    expectation_suite_name="power_gen_exp"
)

expectation_suite.add_expectation(
    ge.core.ExpectationConfiguration(
        expectation_type="expect_column_values_to_be_between",
        kwargs={
            "column": "nuclear",
            "min_value": 0
        }
    )
)

expectation_suite.add_expectation(
    ge.core.ExpectationConfiguration(
        expectation_type="expect_column_values_to_be_between",
        kwargs={
            "column": "solar",
            "min_value": 0
        }
    )
)

## Attach expectation suite to the feature group
se_power_gen_fg.save_expectation_suite(expectation_suite, run_validation=True, validation_ingestion_policy="STRICT")

Attached expectation suite to featuregroup, edit it at https://2a5f8040-2d0d-11ed-b5c5-c151c2fe58c1.cloud.hopsworks.ai:443/p/7287/fs/7235/fg/14352


In [ ]:
se_power_gen_fg.validate(save_report=True)

2022-11-09 09:21:07,378 INFO: USE `electricity_featurestore`
2022-11-09 09:21:08,496 INFO: SELECT `fg0`.`mtu` `mtu`, `fg0`.`fossil_gas` `fossil_gas`, `fg0`.`hydro_water_reservoir` `hydro_water_reservoir`, `fg0`.`marine` `marine`, `fg0`.`nuclear` `nuclear`, `fg0`.`other` `other`, `fg0`.`solar` `solar`, `fg0`.`wind_onshore` `wind_onshore`, `fg0`.`zone` `zone`
FROM `electricity_featurestore`.`se_power_generation_1` `fg0`


2022-11-09 09:21:13,489 INFO: 	2 expectation(s) included in expectation_suite.
Validation Report saved successfully, explore a summary at https://2a5f8040-2d0d-11ed-b5c5-c151c2fe58c1.cloud.hopsworks.ai:443/p/7287/fs/7235/fg/14352


ValidationReport(success: True, {'evaluated_expectations': 2, 'successful_expectations': 2, 'unsuccessful_expectations': 0, 'success_percent': 100.0}, 2 results , {'great_expectations_version': '0.14.3', 'expectation_suite_name': 'power_gen_exp', 'run_id': {'run_name': None, 'run_time': '2022-11-09T14:21:13.488450+00:00'}, 'batch_kwargs': {'ge_batch_id': 'c3ed57fa-6039-11ed-907c-9cb6d0d342bd'}, 'batch_markers': {}, 'batch_parameters': {}, 'validation_time': '20221109T142113.488278Z', 'expectation_suite_meta': {'great_expectations_version': '0.14.3'}}, /Projects/electricity/DataValidation/se_power_generation/1/ValidationReports/validation_report_2022-11-09T142921.json)

In [35]:
wrong_df = se3_power_gen_df.iloc[[0]]

In [37]:
wrong_df['Nuclear'] = -1.0

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [42]:
se_power_gen_fg.insert(wrong_df)

2022-11-09 09:32:28,235 INFO: 	2 expectation(s) included in expectation_suite.
Validation Report saved successfully, explore a summary at https://2a5f8040-2d0d-11ed-b5c5-c151c2fe58c1.cloud.hopsworks.ai:443/p/7287/fs/7235/fg/14352


(None,
 {
   "statistics": {
     "evaluated_expectations": 2,
     "successful_expectations": 1,
     "unsuccessful_expectations": 1,
     "success_percent": 50.0
   },
   "evaluation_parameters": {},
   "meta": {
     "great_expectations_version": "0.14.3",
     "expectation_suite_name": "power_gen_exp",
     "run_id": {
       "run_name": null,
       "run_time": "2022-11-09T14:32:28.234955+00:00"
     },
     "batch_kwargs": {
       "ge_batch_id": "561b98a2-603b-11ed-907c-9cb6d0d342bd"
     },
     "batch_markers": {},
     "batch_parameters": {},
     "validation_time": "20221109T143228.234860Z",
     "expectation_suite_meta": {
       "great_expectations_version": "0.14.3"
     }
   },
   "results": [
     {
       "expectation_config": {
         "meta": {
           "expectationId": 1028
         },
         "expectation_type": "expect_column_values_to_be_between",
         "kwargs": {
           "column": "nuclear",
           "min_value": 0
         }
       },
       "excep